In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import random
import gc
import lightgbm as lgb
import joblib
from lightgbm import LGBMRegressor
from hyperopt import hp, tpe, fmin, Trials, rand, anneal

In [2]:
%%time
base_path = "/kaggle/input/m5-forecasting-accuracy/"
calendar = pd.read_csv(f"{base_path}calendar.csv")
train_eva = pd.read_csv(f"{base_path}sales_train_evaluation.csv")
sell_prices = pd.read_csv(f"{base_path}sell_prices.csv")
sample_sub = pd.read_csv(f"{base_path}sample_submission.csv")

CPU times: user 10.1 s, sys: 1.24 s, total: 11.3 s
Wall time: 14.8 s


In [3]:
# Add more columns in file train
for d in range(1942,1970):
    col = 'd_' + str(d)
    train_eva[col] = 0
    train_eva[col] = train_eva[col].astype(np.int16)

In [4]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

In [5]:
%%time
print("Downcasting data")
train_eva = downcast(train_eva)
sell_prices = downcast(sell_prices)
calendar = downcast(calendar)

Downcasting data
CPU times: user 1min 34s, sys: 1min 25s, total: 2min 59s
Wall time: 3min


In [6]:
%%time
print("Melting data")
df = pd.melt(frame=train_eva, 
             id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
             var_name="d", value_name="sold")

Melting data
CPU times: user 6.55 s, sys: 3.4 s, total: 9.95 s
Wall time: 9.96 s


In [7]:
%%time
print("Merging data")
df = pd.merge(left=df, right=calendar, how="left", on="d")
df = pd.merge(left=df, right=sell_prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")

Merging data
CPU times: user 35.9 s, sys: 9.54 s, total: 45.5 s
Wall time: 45.5 s


In [8]:
%%time
print("Implement features")
#Calculate the SNAP (Supplemental Nutrition Assistance Program) day for each state
df["snap"] = df["snap_CA"] + df["snap_TX"] + df["snap_WI"]
df["snap"] = np.where(df["snap"] >= 1, 1, 0).astype(np.int8)

# Apply int for day column
df["d"] = df["d"].str[2:].astype(np.int16)

# Process NaN value
df["sell_price"] = df['sell_price'].fillna(df.groupby('id')['sell_price'].transform('median'))

# Is it a weekend
df["weekend"] = np.where(df["wday"] < 3, 1, 0).astype(np.int8)

# Drop unnecessary columns
df = df.drop(["date", "weekday", "wm_yr_wk", "event_name_2", "event_type_2", "snap_CA", "snap_TX", "snap_WI"], axis=1)

Implement features
CPU times: user 42.6 s, sys: 3.88 s, total: 46.4 s
Wall time: 46.4 s


In [9]:
# Label Encoder
print("Label Encoding")
d_id = dict(zip(df["id"].cat.codes, df["id"]))
d_store = dict(zip(df["store_id"].cat.codes, df["store_id"]))
df["id"] = df["id"].cat.codes
df["item_id"] = df["item_id"].cat.codes
df["dept_id"] = df["dept_id"].cat.codes
df["cat_id"] = df["cat_id"].cat.codes
df["store_id"] = df["store_id"].cat.codes
df["state_id"] = df["state_id"].cat.codes
df["event_name_1"] = df["event_name_1"].cat.codes
df["event_type_1"] = df["event_type_1"].cat.codes

Label Encoding


In [10]:
%%time
print("Calulating Lags and Rolling mean")
# Lags must be > 28
lags = [29,30,31,32,33,34,35,40,55,60,65,180]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)
    
df['rolling_mean_7']   = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(7).mean())
df['rolling_mean_14']   = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(14).mean())
df['rolling_mean_30']  = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(30).mean())
df['rolling_mean_60']  = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(60).mean())
df['rolling_mean_180']  = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(180).mean())

Calulating Lags and Rolling mean
CPU times: user 5min 7s, sys: 43.4 s, total: 5min 50s
Wall time: 5min 51s


In [11]:
df = df[df["d"] > 28+180]

In [12]:
# Save dataframe
df.to_pickle('data.pkl')
del df
gc.collect()

0

In [13]:
%%time
data = pd.read_pickle('data.pkl')
valid = data[(data['d']>=1914) & (data['d']<1942)][['id','d','sold']]
test = data[data['d']>=1942][['id','d','sold']]

CPU times: user 338 ms, sys: 10.8 s, total: 11.2 s
Wall time: 11.2 s


In [14]:
# Get from file Tuning HyperParams
best_params_store = (
{'CA_1': {'colsample_bytree': 0.8,
  'learning_rate': 0.2,
  'max_depth': 7.0,
  'min_child_weight': 300.0,
  'num_leaves': 200.0,
  'subsample': 0.9},
 'CA_2': {'colsample_bytree': 0.9,
  'learning_rate': 0.30000000000000004,
  'max_depth': 8.0,
  'min_child_weight': 200.0,
  'num_leaves': 200.0,
  'subsample': 0.8},
 'CA_3': {'colsample_bytree': 0.9,
  'learning_rate': 0.2,
  'max_depth': 8.0,
  'min_child_weight': 300.0,
  'num_leaves': 250.0,
  'subsample': 0.9},
 'CA_4': {'colsample_bytree': 0.9,
  'learning_rate': 0.30000000000000004,
  'max_depth': 7.0,
  'min_child_weight': 300.0,
  'num_leaves': 200.0,
  'subsample': 0.9},
 'TX_1': {'colsample_bytree': 0.8,
  'learning_rate': 0.2,
  'max_depth': 7.0,
  'min_child_weight': 300.0,
  'num_leaves': 200.0,
  'subsample': 1.0},
 'TX_2': {'colsample_bytree': 1.0,
  'learning_rate': 0.30000000000000004,
  'max_depth': 8.0,
  'min_child_weight': 300.0,
  'num_leaves': 200.0,
  'subsample': 0.9},
 'TX_3': {'colsample_bytree': 0.8,
  'learning_rate': 0.2,
  'max_depth': 7.0,
  'min_child_weight': 300.0,
  'num_leaves': 250.0,
  'subsample': 0.9},
 'WI_1': {'colsample_bytree': 0.9,
  'learning_rate': 0.2,
  'max_depth': 8.0,
  'min_child_weight': 400.0,
  'num_leaves': 250.0,
  'subsample': 0.8},
 'WI_2': {'colsample_bytree': 1.0,
  'learning_rate': 0.2,
  'max_depth': 8.0,
  'min_child_weight': 200.0,
  'num_leaves': 250.0,
  'subsample': 0.9},
 'WI_3': {'colsample_bytree': 0.9,
  'learning_rate': 0.2,
  'max_depth': 8.0,
  'min_child_weight': 400.0,
  'num_leaves': 250.0,
  'subsample': 0.9}})

In [15]:
%%time
data = pd.read_pickle('data.pkl')
valid = data[(data['d']>=1914) & (data['d']<1942)][['id','d','sold']]
test = data[data['d']>=1942][['id','d','sold']]

CPU times: user 323 ms, sys: 4.31 s, total: 4.64 s
Wall time: 4.65 s


In [16]:
def cross_validation(data, valid_first_day, params, day_start, d_store=d_store, sample_sub=sample_sub):
    
    valid = data[(data['d']>=valid_first_day) & (data['d']<valid_first_day+28)][['id','d','sold']]
    print(f"Valid first day {valid_first_day} predicting")
    for i in range(10): 
        # Forecast cho từng store 
        df = data[data["store_id"] == i]

        #Create train set
        X_train, y_train = df[(df['d']>=day_start) & (df['d']<valid_first_day)].drop('sold',axis=1), df[(df['d']>=day_start) & (df['d']<valid_first_day)]['sold']
        train_sets = lgb.Dataset(X_train, y_train)
        X_valid, y_valid = df[(df['d']>=valid_first_day) & (df['d']<valid_first_day+28)].drop('sold',axis=1), df[(df['d']>=valid_first_day) & (df['d']<valid_first_day+28)]['sold']
        valid_sets = lgb.Dataset(X_valid, y_valid)

        model = lgb.train(params={'objective' : 'tweedie',
                                  'force_row_wise': True,
                                  'verbose': -1,
                                  'n_estimators': 1000,
                                  'learning_rate':params[d_store[i]]["learning_rate"],
                                  'subsample': params[d_store[i]]["subsample"],
                                  'colsample_bytree':params[d_store[i]]["colsample_bytree"],
                                  'min_child_weight':params[d_store[i]]["min_child_weight"],
                                  'max_depth':np.int16(params[d_store[i]]["max_depth"]),
                                  'num_leaves':np.int16(params[d_store[i]]["num_leaves"])},
                                
                      train_set=train_sets, 
                      valid_sets=valid_sets,
                      verbose_eval=False,
                      early_stopping_rounds=50)

        pred_val = model.predict(X_valid)
        valid.loc[X_valid.index, "sold"] = pred_val

    valid["id"] = valid["id"].map(d_id)
    valid = valid.pivot(index="id", columns="d", values="sold").reset_index()
    valid["id"] = valid["id"].str.replace("evaluation", "validation")
    
    sample_sub = sample_sub[["id"]]

    f_col = [f"F{i}" for i in range(1,29)]
    f_col.insert(0, "id")
    
    print(f"Valid testset from day {valid.columns[1]} to day {valid.columns[-1]}")
    
    out_val = pd.merge(left=sample_sub[:30490], right=valid, on="id")
    out_val.columns=f_col
    
    return out_val

def avg_rmsse_score(out_val, train_eva, valid_first_day, day_start):
    
    print(f"Scoring from {valid_first_day} to {valid_first_day+28-1}")
    print(f"Naive first day {day_start}")
    
    days_train = [i for i in range(day_start, valid_first_day)]
    days_valid = [i for i in range(valid_first_day, valid_first_day+28)]
    
    naive_predict = np.array(train_eva[days_train].drop(valid_first_day-1, axis=1)).astype(np.int32)
    y_true_naive = np.array(train_eva[days_train].drop(day_start, axis=1)).astype(np.int32)
    naive_mse = np.mean((naive_predict - y_true_naive) ** 2, axis=1)
    
    model_pred = np.array(out_val.iloc[:, 1:])
    y_true_model = np.array(train_eva[days_valid])
    model_mse = np.mean((model_pred - y_true_model) ** 2, axis=1)
    
    avg_rmsse = np.sqrt(model_mse / naive_mse).mean()
    
    return avg_rmsse

In [17]:
train_eva.columns = list(train_eva.columns[:6]) + [i for i in range(1, 1970)]

In [18]:
valid_first_days_list = [1858, 1886, 1914]
cv_score = dict()

In [19]:
%%time
valid_first_days_list = [1858, 1886, 1914]
day_start = 209
day_start_naive = 1
cv_score = dict()
for i in valid_first_days_list:
    out_df_cv = cross_validation(data=data, valid_first_day=i, day_start=day_start, params=best_params_store)
    cv_score[i] = avg_rmsse_score(out_val=out_df_cv, train_eva=train_eva, valid_first_day=i, day_start=day_start_naive)
    day_start += 28
    day_start_naive += 28

Valid first day 1858 predicting
Valid testset from day 1858 to day 1885
Scoring from 1858 to 1885
Naive first day 1
Valid first day 1886 predicting
Valid testset from day 1886 to day 1913
Scoring from 1886 to 1913
Naive first day 29
Valid first day 1914 predicting
Valid testset from day 1914 to day 1941
Scoring from 1914 to 1941
Naive first day 57
CPU times: user 1h 34min 40s, sys: 1min 56s, total: 1h 36min 37s
Wall time: 30min 10s


In [20]:
for i in valid_first_days_list:
    print(f"Score {i}", cv_score[i])
    
print("CV score", np.mean(list(cv_score.values())))

Score 1858 0.9123814090623443
Score 1886 0.8975892701527566
Score 1914 0.9106487788471872
CV score 0.9068731526874294


In [21]:
%%time
data = pd.read_pickle('data.pkl')
valid = data[(data['d']>=1914) & (data['d']<1942)][['id','d','sold']]
test = data[data['d']>=1942][['id','d','sold']]

CPU times: user 313 ms, sys: 28.2 s, total: 28.5 s
Wall time: 1min 7s


In [22]:
%%time
for i in range(10): 
    # Forecast cho từng store 
    df = data[data["store_id"] == i]
    
    #Create train set
    X_train, y_train = df[df['d']<1914].drop('sold',axis=1), df[df['d']<1914]['sold']
    train_sets = lgb.Dataset(X_train, y_train)
    X_valid, y_valid = df[(df['d']>=1914) & (df['d']<1942)].drop('sold',axis=1), df[(df['d']>=1914) & (df['d']<1942)]['sold']
    valid_sets = lgb.Dataset(X_valid, y_valid)
    X_test = df[df["d"] >= 1942].drop("sold", axis=1)
    
    # Create model
    print(f"Train model for store {d_store[i]}")
    print("--------")

    model = lgb.train(params={'objective' : 'tweedie',
                              'force_row_wise': True,
                              'verbose': -1,
                              'n_estimators': 1000,
                              'learning_rate':best_params_store[d_store[i]]["learning_rate"],
                              'subsample': best_params_store[d_store[i]]["subsample"],
                              'colsample_bytree':best_params_store[d_store[i]]["colsample_bytree"],
                              'min_child_weight':best_params_store[d_store[i]]["min_child_weight"],
                              'max_depth':np.int16(best_params_store[d_store[i]]["max_depth"]),
                              'num_leaves':np.int16(best_params_store[d_store[i]]["num_leaves"])}, 
                      train_set=train_sets, 
                      valid_sets=valid_sets,
                      verbose_eval=50,
                      early_stopping_rounds=50)
    
    print("--------")
    print(f"Predicting for store {d_store[i]}")
    
    # Validation predict
    pred_val = model.predict(X_valid)
    valid.loc[X_valid.index, "sold"] = pred_val
    pred_eva = model.predict(X_test)
    test.loc[X_test.index, "sold"] = pred_eva
    
    print("--------")
    print("Saving model and clear memories")  
    print("--------")
    
    filename = f'model_store_{d_store[i]}.pkl'
    joblib.dump(model, filename)
    del model, X_train, y_train, X_valid, y_valid, X_test, train_sets, valid_sets
    gc.collect 

Train model for store CA_1
--------
Training until validation scores don't improve for 50 rounds
[50]	valid_0's tweedie: 4.29306
[100]	valid_0's tweedie: 4.29036
[150]	valid_0's tweedie: 4.29007
Early stopping, best iteration is:
[118]	valid_0's tweedie: 4.28749
--------
Predicting for store CA_1
--------
Saving model and clear memories
--------
Train model for store CA_2
--------
Training until validation scores don't improve for 50 rounds
[50]	valid_0's tweedie: 4.36271
Early stopping, best iteration is:
[20]	valid_0's tweedie: 4.35669
--------
Predicting for store CA_2
--------
Saving model and clear memories
--------
Train model for store CA_3
--------
Training until validation scores don't improve for 50 rounds
[50]	valid_0's tweedie: 4.85149
[100]	valid_0's tweedie: 4.85041
[150]	valid_0's tweedie: 4.85232
Early stopping, best iteration is:
[104]	valid_0's tweedie: 4.85014
--------
Predicting for store CA_3
--------
Saving model and clear memories
--------
Train model for store C

In [23]:
#model = joblib.load("/kaggle/working/model_store_CA_1.pkl")
#lgb.plot_importance(model, importance_type="gain", figsize=(15,8))

In [24]:
valid["id"] = valid["id"].map(d_id)
valid = valid.pivot(index="id", columns="d", values="sold").reset_index()
valid["id"] = valid["id"].str.replace("evaluation", "validation")

sample_sub = sample_sub[["id"]]

f_col = [f"F{i}" for i in range(1,29)]
f_col.insert(0, "id")

out_val = pd.merge(left=sample_sub[:30490], right=valid, on="id")
out_val.columns=f_col

test["id"] = test["id"].map(d_id)
test = test.pivot(index="id", columns="d", values="sold").reset_index()

out_eva = pd.merge(left=sample_sub[30490:], right=test, on="id")
out_eva.columns=f_col

submit = pd.concat([out_val,out_eva], ignore_index=True)

In [25]:
submit

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.920875,0.760546,0.757955,0.711162,0.828706,1.097972,1.029628,0.782116,0.771163,...,0.965307,1.217575,1.253032,0.949194,0.776976,0.754316,0.821256,0.919370,1.157267,1.143255
1,HOBBIES_1_002_CA_1_validation,0.380173,0.345906,0.326023,0.320066,0.366852,0.457072,0.417306,0.309664,0.265039,...,0.309420,0.445289,0.391920,0.304422,0.258979,0.242773,0.243011,0.281972,0.313900,0.258163
2,HOBBIES_1_003_CA_1_validation,0.564305,0.463523,0.446733,0.490597,0.567147,0.679917,0.638504,0.450915,0.444713,...,0.604698,0.730895,0.736315,0.620832,0.588095,0.631331,0.587575,0.695829,0.845153,0.874448
3,HOBBIES_1_004_CA_1_validation,1.943717,1.705169,1.613644,1.512644,1.807105,2.570215,2.556299,1.831783,1.535107,...,1.633205,2.468217,2.359394,1.754184,1.527192,1.558596,1.424226,1.664176,2.356025,2.691174
4,HOBBIES_1_005_CA_1_validation,1.085582,0.805124,0.994593,1.075440,1.204261,1.460079,1.508997,1.184461,1.091212,...,1.022125,1.350813,1.397258,0.984482,0.870272,0.861293,0.847483,1.174094,1.412847,1.572596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.384847,0.408038,0.399819,0.446521,0.669699,0.741491,0.714729,0.493718,0.545130,...,0.716556,0.812492,0.818664,0.707584,0.669967,0.687489,0.541193,0.668842,0.658130,0.803895
60976,FOODS_3_824_WI_3_evaluation,0.270022,0.279047,0.287140,0.382968,0.445324,0.426837,0.386212,0.315894,0.361767,...,0.256865,0.288135,0.288428,0.264014,0.304923,0.301380,0.318282,0.309472,0.407646,0.374816
60977,FOODS_3_825_WI_3_evaluation,0.591817,0.572444,0.552542,0.585518,0.668301,0.906825,0.881910,0.703805,0.618025,...,1.110056,1.190463,1.152983,1.174852,1.085875,0.930320,0.695539,0.842074,1.003697,0.969664
60978,FOODS_3_826_WI_3_evaluation,0.940807,0.920369,0.996659,0.993060,1.099301,1.351013,1.253780,0.974163,0.892712,...,1.291240,1.706234,1.398037,1.310340,1.664076,1.605405,1.415688,1.467283,1.614603,1.315356


In [26]:
submit.to_csv('submission.csv',index=False)